In [ ]:
# import importlib
# importlib.reload(franky)

In [1]:
import franky
import numpy as np
np.set_printoptions(precision=6, suppress=True, linewidth=200)

from scipy.spatial.transform import Rotation

In [2]:
robot = franky.Robot("172.168.0.2", realtime_config=franky.RealtimeConfig.Ignore)

In [ ]:
robot.forward_kinematics(np.array([0, 0, 0, 0, 0, 0, 0]))

In [4]:
pos = robot.forward_kinematics(np.array(robot.state.q))

In [ ]:
%%timeit -r 10 -n 100  q0 = np.array(robot.state.q) + np.random.randn(7) * 0.1
q_result = robot.inverse_kinematics(pos, q0)

In [ ]:
%%time
q0 = np.array(robot.state.q) + np.random.randn(7) * 0.1
robot.inverse_kinematics(pos, q0)

In [5]:
def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2

    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

    return np.array([w, x, y, z])

In [6]:
def q_inv(q):
    q = q.copy()
    q[1:] = -q[1:]
    return q

---

In [30]:
def error(x_cur, target):
    pos_error = x_cur.translation - target.translation
    R_cur = x_cur.matrix[:3,:3]  # Current rotation matrix
    R_target = target.matrix[:3,:3]  # Target rotation matrix
    R_error = R_cur.T @ R_target
    rot = Rotation.from_matrix(R_error)
    rot_vec = rot.as_rotvec()
    rot_vec = -R_cur @ rot_vec
    return np.concatenate([pos_error, rot_vec])

In [9]:
# _q = np.zeros(7)
# _q = np.array([0, np.pi/2, 0, np.pi/2, 0, 0, 0])
_q = np.array([0, 0, 0, 0, 0, 0, 0])

In [10]:
eps = 1e-5
J_num = np.zeros((6, 7))

for i in range(7):
    dq = np.zeros(7)
    dq[i] = eps
    x_plus = error(robot.forward_kinematics(_q + dq), robot.forward_kinematics(_q))
    x_minus = error(robot.forward_kinematics(_q - dq), robot.forward_kinematics(_q))
    J_num[:,i] = (x_plus - x_minus) / (2 * eps)

In [ ]:
J_num

In [ ]:
J = robot.jacobian(_q)
J

---

In [31]:
def line_search(robot, q, dq, e_current, target, alpha=1.0, beta=0.5, max_steps=10):
    current_error_norm = np.linalg.norm(e_current)
    step_size = alpha

    for _ in range(max_steps):
        q_new = q + step_size * dq
        x_new = robot.forward_kinematics(q_new)
        e_new = error(x_new, target)

        if np.linalg.norm(e_new) < current_error_norm:
            return step_size

        step_size *= beta

    return step_size

In [86]:
def inverse_kinematics(robot, target: franky.Affine, q0):
    q = q0.copy()
    I = np.eye(7)
    k0 = 0.01
    max_iterations = 50
    tol = 1e-4; min_step = 1e-6
    pinv_reg = 0.1

    for i in range(max_iterations):
        x = robot.forward_kinematics(q)
        e = error(x, target)
        error_norm = np.linalg.norm(e)

        if error_norm < tol:
            print(f"Converged after {i} iterations")
            break

        j = robot.jacobian(q)
        j_inv = j.T @ np.linalg.inv(j @ j.T + pinv_reg * I[:6,:6])

        N = I - j_inv @ j
        dq_primary = -j_inv @ e

        k_null = k0 * np.exp(error_norm)  # Reduce null space influence when error is high
        dq_null = N @ (-k_null * q)

        dq = dq_primary + dq_null

        step_size = line_search(robot, q, dq, e, target, alpha=1.0, beta=0.8, max_steps=20)
        if step_size < min_step:
            print(f"Step size too small after {i} iterations")
            break

        q += step_size * dq

        if i % 1 == 0:  # Print less frequently
            print(f"{i}: Err: {error_norm:.5f}, Step: {step_size:.3f}, dq_p: {np.linalg.norm(dq_primary)}, dq_n: {np.linalg.norm(dq_null)}")

    return q

In [87]:
pos = robot.forward_kinematics(np.array(robot.state.q))

In [ ]:
ik_q = inverse_kinematics(robot, pos, np.array(robot.state.q) + np.random.randn(7) * 0.06)
robot.forward_kinematics(ik_q), robot.forward_kinematics(np.array(robot.state.q))

In [ ]:
np.linalg.norm(ik_q), np.linalg.norm(np.array(robot.state.q))

In [ ]:
np.linalg.norm(ik_q), np.linalg.norm(np.array(robot.state.q))

In [12]:
target = pos
q0 = np.array(robot.state.q) + np.random.randn(7) * 0.03
q = q0.copy()
pinv_reg, I = 0.01, np.eye(7)

In [13]:
x = robot.forward_kinematics(q)
e = error(x, target)
error_norm = np.linalg.norm(e)

In [8]:
def error(x_cur, target):
    quat = np.array(x_cur.quaternion)
    q_e = quaternion_multiply(quat, q_inv(target.quaternion))
    q_e = Rotation.from_quat([q_e[1], q_e[2], q_e[3], q_e[0]]).as_euler('ZYX')
    return np.concatenate([x_cur.translation - target.translation, q_e])

In [29]:
# _q = np.zeros(7)
_q = np.array([0, np.pi/2, 0, np.pi/2, 0, 0, 0])
_q = robot.state.q

In [30]:
eps = 1e-5
J_num = np.zeros((6, 7))

for i in range(7):
    dq = np.zeros(7)
    dq[i] = eps
    x_plus = error(robot.forward_kinematics(_q + dq), robot.forward_kinematics(_q))
    x_minus = error(robot.forward_kinematics(_q - dq), robot.forward_kinematics(_q))
    J_num[:,i] = (x_plus - x_minus) / (2 * eps)

In [ ]:
J_num

In [ ]:
J = robot.jacobian(_q)
J

In [36]:
q1 = _q + 0.01 * np.array([0, 1, 0, 0, 0, 0, 0])

In [38]:
p1, p = robot.forward_kinematics(q1), robot.forward_kinematics(_q)
# np.array(p1.quaternion), np.array(p.quaternion)
# np.array(p1.quaternion) - np.array(p.quaternion)

In [ ]:
p1, p

In [ ]:
p1.translation - p.translation

In [ ]:
tr_diff = quaternion_multiply(np.array(p1.quaternion), q_inv(p.quaternion))
tr_diff

In [ ]:
Rotation.from_quat([tr_diff[1], tr_diff[2], tr_diff[3], tr_diff[0]]).as_euler("ZYX")


In [ ]:
error(p1, p)

In [ ]:
np.array(J @ (q1 - _q))

In [ ]:
Rotation.from_euler("ZYX", np.array(J @ (q1 - _q))[3:]).as_quat()

In [ ]:
p.translation + np.array(J @ (q1 - _q))[:3], p1.translation

In [ ]:
Rotation.from_quat(tr_diff).as_euler("ZYX", degrees=True)


In [ ]:
j

In [ ]:
j_inv @ e

---

In [13]:
pos = robot.current_cartesian_state.pose.end_effector_pose
q = np.array(robot.state.q)

In [ ]:
pos

In [ ]:
a = np.array(robot.state.O_T_EE)
franky.Affine(a.reshape(4,4).T)

In [ ]:
robot.forward_kinematics(q)

In [ ]:
inverse_kinematics(robot, pos, np.array(robot.state.q))

In [ ]:
robot.forward_kinematics(robot.state.q)

In [ ]:
robot.state.q